In [25]:
import pandas as pd
import re

from nltk import PorterStemmer, SnowballStemmer
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.metrics import confusion_matrix,classification_report,accuracy_score

import warnings
warnings.filterwarnings('ignore')


col = ['target', 'id', 'date', 'flag', 'user', 'text']

df = pd.read_csv('/content/sample_data/training.1600000.processed.noemoticon.csv', header = None, names = col,  encoding='latin-1')

# remove empty
df.dropna()
df.fillna(value='', inplace=True)


# df = df[['text','target']]
df = df.sample(n=15000)

# Handle Categories variable
df['target'] = df['target'].replace(4,1)


df['text'].head()

662662                                            homework 
313981      @missdolan orangizing??? why am i not with you 
257298    I hate Actionscript, it doesn't like me neithe...
490729    @mroneof yes shit was crazy. I can't find my c...
923816                             4 points from the title 
Name: text, dtype: object

In [26]:

def get_part_of_day(h):
        if   5 <= h <= 11:
            return 'morning'
            # return '0'
        elif 12 <= h <= 17:
            return "afternoon"
            # return "1"
        elif 18 <= h <= 22:
            return 'evening'
            # return '2'
        else:
            return "night"
            # return "3"


day = []
month = []
dayInMonth = []
timeOfTweet = []
timePeriod = []
year = []

for d in df['date']:
    sub = d.split(' ')
    # day.append(dayInMonth(sub[0]))
    day.append(sub[0])
    # month.append(sub[1])
    dayInMonth.append(sub[2])
    # timeOfTweet.append(sub[3])
    timePeriod.append(get_part_of_day(int(sub[3][:2])))

    # time zones is only pdt
    # timeZone.append(sub[4])

    # year is only 2009
    # year.append(sub[5])



data = pd.DataFrame()

data['day'] = day
# data['month'] = month
data['dayInMonth'] = dayInMonth
# data['time'] = time
data['partOfDay'] = timePeriod

data = pd.get_dummies(data, columns=['day','partOfDay'])



In [27]:
data['target'] = df['target'].values
data['text'] = df['text'].values

df = data

data.head()

,dayInMonth,day_Fri,day_Mon,day_Sat,day_Sun,day_Thu,day_Tue,day_Wed,partOfDay_afternoon,partOfDay_evening,partOfDay_morning,partOfDay_night,target,text
0,19,1,0,0,0,0,0,0,1,0,0,0,0,homework
1,02,0,0,0,0,0,1,0,0,0,0,1,0,@missdolan orangizing??? why am i not with you
2,31,0,0,0,1,0,0,0,1,0,0,0,0,"I hate Actionscript, it doesn't like me neithe..."
3,15,0,1,0,0,0,0,0,1,0,0,0,0,@mroneof yes shit was crazy. I can't find my c...
4,10,0,0,0,1,0,0,0,0,0,1,0,1,4 points from the title


In [28]:
print("starting text clean")


# !pip install langdetect
# from langdetect import detect


ps = PorterStemmer()
sb = SnowballStemmer(language='english')


import nltk
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('omw-1.4')


all_stopwords = stopwords.words('english')

corpus = []
corpus1 = []



for sen in df['text']:
    # remove hashtags
    sen = re.sub("(@[A-Za-z0-9_]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)", ' ', sen)
#     sen = re.sub('(@|#|&)+([a-zA-Z0-9_]+)', '', sen)
    # remove weird chars
    sen = re.sub('[^a-zA-z\'\"]+', ' ', sen)
    # remove urls
    sen = re.sub(r'\$\w*', '', sen)
    # remove old style retweet text "RT"
    sen = re.sub(r'^RT[\s]+', '', sen)
    # remove hyperlinks
    sen = re.sub(r'https?:\/\/.*[\r\n]*', '', sen)
    # only removing the hash # sign from the word
    sen = re.sub(r'#', '', sen)

    # if sen == "" or sen.isspace():
    #         continue


    # if detect(sen) != 'en':
    #         continue

    sen = sen.lower()
    sen = sen.split()
    
    
    text1 = [ps.stem(word) for word in sen if not word in set(all_stopwords)]
#     text1 = [sb.stem(word) for word in sen if not word in set(all_stopwords)]
    
#     text = ' '.join(sen)
    text1 = ' '.join(text1)
    
    
#     corpus.append(text)
    corpus1.append(text1)


starting text clean


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [29]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer()
vector = vectorizer.fit_transform(corpus1)

In [30]:
print(type(vector))
words = pd.DataFrame.sparse.from_spmatrix(vector)
print(type(words))
df = pd.concat([df, words], axis=1, join='inner')


df = df.drop('text',axis=1)

df[df.columns[15:]] = df.iloc[:, 15:].astype('float16')


<class 'scipy.sparse.csr.csr_matrix'>
<class 'pandas.core.frame.DataFrame'>


In [31]:
y = df.target
X = df.drop('target', axis=1)
X1 = pd.DataFrame.sparse.from_spmatrix(vector)

X.head()

,dayInMonth,day_Fri,day_Mon,day_Sat,day_Sun,day_Thu,day_Tue,day_Wed,partOfDay_afternoon,partOfDay_evening,...,13164,13165,13166,13167,13168,13169,13170,13171,13172,13173
0,19,1,0,0,0,0,0,0,1,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,02,0,0,0,0,0,1,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,31,0,0,0,1,0,0,0,1,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,15,0,1,0,0,0,0,0,1,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,10,0,0,0,1,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [32]:
X1.head()

,0,1,2,3,4,5,6,7,8,9,...,13164,13165,13166,13167,13168,13169,13170,13171,13172,13173
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


##  Only text vectorization

In [33]:
print("starting splitting data")

X_train, X_test, y_train, y_test = train_test_split(X, y,test_size = 0.3, random_state = 42)
X_train1, X_test1, y_train1, y_test1 = train_test_split(X1, y,test_size = 0.3, random_state = 42)


starting splitting data


In [34]:
# models
# !pip install sklearn.naive_baye
# !pip install sklearn

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier,AdaBoostClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import BaggingClassifier
from sklearn.linear_model import LogisticRegression

DecTr = DecisionTreeClassifier()
AdaBoost = AdaBoostClassifier()
Knn = KNeighborsClassifier()
# svm = SVC(gamma='auto')
# mlp = MLPClassifier()
# gpc = GaussianProcessClassifier()
# gnb = GaussianNB()
rfc = RandomForestClassifier()
bc = BaggingClassifier()
lr = LogisticRegression()

DecTr1 = DecisionTreeClassifier()
AdaBoost1 = AdaBoostClassifier()
Knn1 = KNeighborsClassifier()
# svm1 = SVC(gamma='auto')
# mlp1 = MLPClassifier()
# gpc1 = GaussianProcessClassifier()
# gnb1 = GaussianNB()
rfc1 = RandomForestClassifier()
bc1 = BaggingClassifier()
lr1 = LogisticRegression()

In [11]:
print("fitting extended data")
DecTr.fit(X_train,y_train)
AdaBoost.fit(X_train,y_train)
Knn.fit(X_train,y_train)
# svm.fit(X_train,y_train)
# mlp.fit(X_train,y_train)
# gpc.fit(X_train,y_train)
# gnb.fit(X_train,y_train)
rfc.fit(X_train,y_train)
bc.fit(X_train,y_train)
lr.fit(X_train,y_train)

fitting extended data


LogisticRegression()

In [12]:

# svm1 = SVC(probability=True)
print("fitting only text")
DecTr1.fit(X_train1,y_train1)
AdaBoost1.fit(X_train1,y_train1)
Knn1.fit(X_train1,y_train1)
# X_train1Numpy = vector.toarray()
# svm1.fit(X_train1Numpy,y_train1)
# mlp1.fit(X_train1,y_train1)
# gpc1.fit(X_train1Numpy,y_train1)
# gnb1.fit(X_train1Numpy,y_train1)
rfc1.fit(X_train1,y_train1)
bc1.fit(X_train1,y_train1)
lr1.fit(X_train1,y_train1)

fitting only text


LogisticRegression()

In [13]:
print("predicting extended")
predictionsa = DecTr.predict(X_test)
predictionsb = AdaBoost.predict(X_test)
predictionsc = Knn.predict(X_test)
# predictionsd = svm.predict(X_test)
# predictionse = mlp.predict(X_test)
# predictionsf = gpc.predict(X_test)
# predictionsg = gnb.predict(X_test)
predictionsh = rfc.predict(X_test)
predictionsi = bc.predict(X_test)
predictionsj = lr.predict(X_test)

predicting extended


In [14]:
print("predicting original")
predictions1a = DecTr1.predict(X_test1)
predictions1b = AdaBoost1.predict(X_test1)
predictions1c = Knn1.predict(X_test1)
# predictions1d = svm1.predict(X_test1)
# predictions1e = mlp1.predict(X_test1)
# predictions1f = gpc1.predict(X_test1)
# predictions1g = gnb1.predict(X_test1)
predictions1h = rfc1.predict(X_test1)
predictions1i = bc1.predict(X_test1)
predictions1j = lr1.predict(X_test1)

predicting original


In [15]:
print("Decision tree\n")

print("extended:")
print(confusion_matrix(y_test,predictionsa))
print(classification_report(y_test,predictionsa))
print(accuracy_score(y_test,predictionsa))

print("original:")
print(confusion_matrix(y_test1,predictions1a))
print(classification_report(y_test1,predictions1a))
print(accuracy_score(y_test1,predictions1a))

Decision tree

extended:
[[1585  782]
 [ 685 1748]]
              precision    recall  f1-score   support

           0       0.70      0.67      0.68      2367
           1       0.69      0.72      0.70      2433

    accuracy                           0.69      4800
   macro avg       0.69      0.69      0.69      4800
weighted avg       0.69      0.69      0.69      4800

0.694375
original:
[[1563  804]
 [ 725 1708]]
              precision    recall  f1-score   support

           0       0.68      0.66      0.67      2367
           1       0.68      0.70      0.69      2433

    accuracy                           0.68      4800
   macro avg       0.68      0.68      0.68      4800
weighted avg       0.68      0.68      0.68      4800

0.6814583333333334


In [16]:
print("Adaboost\n")

print("extended:")
print(confusion_matrix(y_test,predictionsb))
print(classification_report(y_test,predictionsb))
print(accuracy_score(y_test,predictionsb))

print("original:")
print(confusion_matrix(y_test1,predictions1b))
print(classification_report(y_test1,predictions1b))
print(accuracy_score(y_test1,predictions1b))

Adaboost

extended:
[[1451  916]
 [ 513 1920]]
              precision    recall  f1-score   support

           0       0.74      0.61      0.67      2367
           1       0.68      0.79      0.73      2433

    accuracy                           0.70      4800
   macro avg       0.71      0.70      0.70      4800
weighted avg       0.71      0.70      0.70      4800

0.7022916666666666
original:
[[ 963 1404]
 [ 275 2158]]
              precision    recall  f1-score   support

           0       0.78      0.41      0.53      2367
           1       0.61      0.89      0.72      2433

    accuracy                           0.65      4800
   macro avg       0.69      0.65      0.63      4800
weighted avg       0.69      0.65      0.63      4800

0.6502083333333334


In [17]:
print("KNN\n")

print("extended:")
print(confusion_matrix(y_test,predictionsc))
print(classification_report(y_test,predictionsc))
print(accuracy_score(y_test,predictionsc))

print("original:")
print(confusion_matrix(y_test1,predictions1c))
print(classification_report(y_test1,predictions1c))
print(accuracy_score(y_test1,predictions1c))

KNN

extended:
[[1332 1035]
 [ 604 1829]]
              precision    recall  f1-score   support

           0       0.69      0.56      0.62      2367
           1       0.64      0.75      0.69      2433

    accuracy                           0.66      4800
   macro avg       0.66      0.66      0.65      4800
weighted avg       0.66      0.66      0.66      4800

0.6585416666666667
original:
[[ 793 1574]
 [ 729 1704]]
              precision    recall  f1-score   support

           0       0.52      0.34      0.41      2367
           1       0.52      0.70      0.60      2433

    accuracy                           0.52      4800
   macro avg       0.52      0.52      0.50      4800
weighted avg       0.52      0.52      0.50      4800

0.5202083333333334


In [18]:
# print("SVM\n")

# print("extended:")
# print(confusion_matrix(y_test,predictionsd))
# print(classification_report(y_test,predictionsd))
# print(accuracy_score(y_test,predictionsd))

# print("original:")
# print(confusion_matrix(y_test1,predictions1d))
# print(classification_report(y_test1,predictions1d))
# print(accuracy_score(y_test1,predictions1d))

In [19]:
# print("MLP\n")

# print("extended:")
# print(confusion_matrix(y_test,predictionse))
# print(classification_report(y_test,predictionse))
# print(accuracy_score(y_test,predictionse))

# print("original:")
# print(confusion_matrix(y_test1,predictions1e))
# print(classification_report(y_test1,predictions1e))
# print(accuracy_score(y_test1,predictions1e))

In [20]:
# print("Gaussian Process\n")

# print("extended:")
# print(confusion_matrix(y_test,predictionsf))
# print(classification_report(y_test,predictionsf))
# print(accuracy_score(y_test,predictionsf))

# print("original:")
# print(confusion_matrix(y_test1,predictions1f))
# print(classification_report(y_test1,predictions1f))
# print(accuracy_score(y_test1,predictions1f))

In [21]:
# print("Gaussian NB\n")

# print("extended:")
# print(confusion_matrix(y_test,predictionsg))
# print(classification_report(y_test,predictionsg))
# print(accuracy_score(y_test,predictionsg))

# print("original:")
# print(confusion_matrix(y_test1,predictions1g))
# print(classification_report(y_test1,predictions1g))
# print(accuracy_score(y_test1,predictions1g))


In [22]:
print("Random Forest\n")

print("extended:")
print(confusion_matrix(y_test,predictionsh))
print(classification_report(y_test,predictionsh))
print(accuracy_score(y_test,predictionsh))

print("original:")
print(confusion_matrix(y_test1,predictions1h))
print(classification_report(y_test1,predictions1h))
print(accuracy_score(y_test1,predictions1h))


Random Forest

extended:
[[1573  794]
 [ 440 1993]]
              precision    recall  f1-score   support

           0       0.78      0.66      0.72      2367
           1       0.72      0.82      0.76      2433

    accuracy                           0.74      4800
   macro avg       0.75      0.74      0.74      4800
weighted avg       0.75      0.74      0.74      4800

0.7429166666666667
original:
[[1602  765]
 [ 550 1883]]
              precision    recall  f1-score   support

           0       0.74      0.68      0.71      2367
           1       0.71      0.77      0.74      2433

    accuracy                           0.73      4800
   macro avg       0.73      0.73      0.73      4800
weighted avg       0.73      0.73      0.73      4800

0.7260416666666667


In [23]:
print("Bagging\n")

print("extended:")
print(confusion_matrix(y_test,predictionsi))
print(classification_report(y_test,predictionsi))
print(accuracy_score(y_test,predictionsi))

print("original:")
print(confusion_matrix(y_test1,predictions1i))
print(classification_report(y_test1,predictions1i))
print(accuracy_score(y_test1,predictions1i))

Bagging

extended:
[[1594  773]
 [ 595 1838]]
              precision    recall  f1-score   support

           0       0.73      0.67      0.70      2367
           1       0.70      0.76      0.73      2433

    accuracy                           0.71      4800
   macro avg       0.72      0.71      0.71      4800
weighted avg       0.72      0.71      0.71      4800

0.715
original:
[[1697  670]
 [ 750 1683]]
              precision    recall  f1-score   support

           0       0.69      0.72      0.71      2367
           1       0.72      0.69      0.70      2433

    accuracy                           0.70      4800
   macro avg       0.70      0.70      0.70      4800
weighted avg       0.70      0.70      0.70      4800

0.7041666666666667


In [24]:
print("Logistic Regression\n")

print("extended:")
print(confusion_matrix(y_test,predictionsj))
print(classification_report(y_test,predictionsj))
print(accuracy_score(y_test,predictionsj))

print("original:")
print(confusion_matrix(y_test1,predictions1j))
print(classification_report(y_test1,predictions1j))
print(accuracy_score(y_test1,predictions1j))


Logistic Regression

extended:
[[1671  696]
 [ 525 1908]]
              precision    recall  f1-score   support

           0       0.76      0.71      0.73      2367
           1       0.73      0.78      0.76      2433

    accuracy                           0.75      4800
   macro avg       0.75      0.75      0.75      4800
weighted avg       0.75      0.75      0.75      4800

0.745625
original:
[[1653  714]
 [ 560 1873]]
              precision    recall  f1-score   support

           0       0.75      0.70      0.72      2367
           1       0.72      0.77      0.75      2433

    accuracy                           0.73      4800
   macro avg       0.74      0.73      0.73      4800
weighted avg       0.74      0.73      0.73      4800

0.7345833333333334
